# Validação de prompt (multi execução)
Este notebook executa um mesmo prompt "n" vezes, para dois diferentes casos de busca, com o intuito de observar o comportamento do prompt em um cenário de variações (chats) e validar sua consistência (coerência e reprodutibilidade).

In [ ]:
# --------------------------------------------------
# Instalar dependências
# --------------------------------------------------

# Instala a versão mais recente da OpenAI (se já estiver ok, pode pular esta linha)
!pip install --quiet --upgrade openai openpyxl pandas

In [ ]:
# --------------------------------------------------
# Algoritmo geral
# --------------------------------------------------

# Importar bibliotecas
from openai import OpenAI
import os
import json
import re
import pandas as pd

# COnfigurações gerais
NUM_RODADAS = 50  # <-- quantas execuções por caso
ARQ_SAIDA_XLSX = "/content/resultados_obda.xlsx"

# ========= Chave de API OpenAI =========

# Com variável de ambiente por segurança:
# os.environ["OPENAI_API_KEY"] = "sk-..."
#client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

# ou

# Com valor direto
client = OpenAI(api_key="")

# Casos de busca
casos_busca = [
    {
        "nome": "Caso_1",
        "expressao": """
mostrar o "id" dos animais cuja espécie é da classe "Giraffe"
""".strip()
    },
    {
        "nome": "Caso_2",
        "expressao": """
mostrar o "id" e a "taxon" dos animais cuja espécie é da classe "Giraffe"
""".strip()
    }
]

# Carregar arquivos JSON
with open("/content/obda_chat_prompts_v3_PT-EN.json", "r", encoding="utf-8") as f:
    json_prompt = json.load(f)

with open("/content/AnimalsDataBase_ADB_schema.json", "r", encoding="utf-8") as f:
    json_dados = json.load(f)

with open("/content/AfricanWildlifeOntology_AWO.json", "r", encoding="utf-8") as f:
    json_ontologia = json.load(f)

# Extrair contexto e regras
contexto_str = json_prompt["versions"]["pt-BR"]["main_prompt"]["context"]
regras_str = "\n".join(json_prompt["versions"]["pt-BR"]["main_prompt"]["rules"]["strict_rules"])

# Helper: parser dos 3 blocos
def extrair_tres_blocos(texto_resposta: str):
    """
    Extrai (SQL, Ontop, SPARQL) nessa ordem, priorizando fenced code blocks (```).
    Possui fallback simples para não retornar vazio.
    """
    # 1) Pegue os 3 primeiros blocos cercados por crases
    code_blocks = re.findall(r"```[a-zA-Z0-9_\-]*\n([\s\S]*?)```", texto_resposta)
    if len(code_blocks) >= 3:
        return code_blocks[0].strip(), code_blocks[1].strip(), code_blocks[2].strip()

    # 2) Seções nomeadas + bloco cercado (mais tolerante)
    def cap(label_regex):
        m = re.search(rf"{label_regex}[\s\S]*?```[a-zA-Z0-9_\-]*\n([\s\S]*?)```",
                      texto_resposta, flags=re.IGNORECASE)
        return m.group(1).strip() if m else ""

    sql    = cap(r"(SQL\s*Query|Consulta\s*SQL)")
    ontop  = cap(r"(Ontop\s*Mapping|Mapeamento\s*Ontop)")
    sparql = cap(r"(SPARQL)")

    # 3) Fallback final: garante que não fiquem vazios
    if not (sql and ontop and sparql):
        partes = re.split(r"\n{2,}", texto_resposta.strip())
        faltas = [k for k, ok in [("sql", sql), ("ontop", ontop), ("sparql", sparql)] if not ok]
        for i, falta in enumerate(faltas):
            trecho = (partes[i].strip() if i < len(partes) else texto_resposta.strip())
            if falta == "sql" and not sql: sql = trecho
            elif falta == "ontop" and not ontop: ontop = trecho
            elif falta == "sparql" and not sparql: sparql = trecho

    return sql, ontop, sparql

# Execução: NUM_RODADAS por caso + XLSX (2 abas)
with pd.ExcelWriter(ARQ_SAIDA_XLSX, engine="openpyxl") as writer:
    for caso in casos_busca:
        linhas = []

        for rodada in range(NUM_RODADAS):
            # Monta as mensagens com o mesmo formato do seu código original
            messages = [
                {"role": "system", "content": contexto_str},
                {
                    "role": "user",
                    "content": f"""
Expressão de busca:
{caso['expressao']}

Descrição da estrutura dos dados:
{json.dumps(json_dados, indent=2, ensure_ascii=False)}

Descrição da ontologia:
{json.dumps(json_ontologia, indent=2, ensure_ascii=False)}

Siga estritamente as seguintes regras:
{regras_str}

Gere os três blocos de código: SQL Query, Ontop Mapping (.obda), SPARQL Query.
""".strip()
                }
            ]

            # Chamada com parâmetros não determinísticos (ambiente de chats)
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                temperature=0.7,
                top_p=0.9,
                frequency_penalty=0.3,
                presence_penalty=0.2,
                max_tokens=1500
            )

            texto = (response.choices[0].message.content or "").strip()
            sql, ontop, sparql = extrair_tres_blocos(texto)

            linhas.append({
                "Rodada": rodada + 1,
                "SQL": sql,
                "Ontop Mapping": ontop,
                "SPARQL": sparql
            })

        # Salva a aba do caso atual
        df = pd.DataFrame(linhas, columns=["Rodada", "SQL", "Ontop Mapping", "SPARQL"])
        df.to_excel(writer, index=False, sheet_name=caso["nome"])

print(f"Arquivo gerado em: {ARQ_SAIDA_XLSX}")

Arquivo gerado em: /content/resultados_obda.xlsx
